In [1]:
# import data

import pandas as pd

monthly_expense = pd.read_excel('https://docs.google.com/spreadsheets/d/' + '10OGbaywwMIqKgnPGy8VDvpBVtjyqln47iYa2lFhI9Mw' + '/export?format=xlsx', sheet_name='Sheet1')
customer_lifespan_data = pd.read_excel('https://docs.google.com/spreadsheets/d/' + '1by8tPHwOnq3uKYK2E7sA9VBUYoPM4p1Rnrm_Ss9cyHI' + '/export?format=xlsx', sheet_name='Sheet1')
daily_marketing_spendings = pd.read_excel('https://docs.google.com/spreadsheets/d/' + '1AZOIThOV4P-0eYDge53ZwumVkfkHoYPWxst3k3Bv87c' + '/export?format=xlsx', sheet_name='Sheet1')
Payroll = pd.read_excel('https://docs.google.com/spreadsheets/d/' + '1c_WihqTZCQvNgxzmd-OwhR9i5diwtfxXVLyMn8R-Lp4' + '/export?format=xlsx', sheet_name='Sheet1')
receipts_history = pd.read_excel('https://docs.google.com/spreadsheets/d/' + '1qayqML1zCKdmtzutkcy9LWvE6xFRm6TGBEVkHHJKIuE' + '/export?format=xlsx', sheet_name='Sheet1')


# 1.CAC

## 1.1. Sales Expense

We will calculate the sales expenses. In this case, it is the expense for **Salesforce Software Licenses** which can be found in the **monthly_expense** dataframe

In [2]:
# Extract last month

last_month = monthly_expense['month'].max().month

# Last month expense

cond = monthly_expense['month'].dt.month == last_month
last_month_expense = monthly_expense[cond]
last_month_expense.head()

,#,month,category,item,amount
18,19,2023-03-01,Server Costs,AWS Hosting,8400
19,20,2023-03-01,Server Costs,Google Cloud Storage,4400
20,21,2023-03-01,Software Licenses,Atlassian Jira,1400
21,22,2023-03-01,Software Licenses,Slack,900
22,23,2023-03-01,Software Licenses,Salesforce,1700


In [3]:
# Sales Expense
cond = last_month_expense['item'] == 'Salesforce'
sales_expense = last_month_expense[cond]['amount'].sum()

print('Total salse expense:', sales_expense)

Total salse expense: 1700


## 1.2. Sales and Marketing Payroll

We will calcualte last month **sales and marketing salaries** which can be found in **Payroll dataframe**

We only calculate the payroll for Sales and Marketing department

In [4]:
# Last month payroll

cond = Payroll['month'].dt.month == last_month
last_month_payroll = Payroll[cond]
last_month_payroll.head()

# Total Sales and Marketing payroll
sales_mkt_dept = ['Sales', 'Marketing']
cond2 = last_month_payroll['department'].isin(sales_mkt_dept)
sales_mkt_payroll = last_month_payroll[cond2]['paid'].sum()
print('Total sales and marketing salary:',sales_mkt_payroll)

Total sales and marketing salary: 5950


## 1.3. Marketing Spending

We will calculate **last month total marketing spending** based on **daily_marketing_spendings** dataframe

In [5]:
# Last month marketing spending

cond = daily_marketing_spendings['date'].dt.month == last_month
last_month_marketing_spendings = daily_marketing_spendings[cond]['spending'].sum()
print('Total marketing spending:',last_month_marketing_spendings)

Total marketing spending: 68830


## 1.4. Total New Customer

We calculate **last month new customers** based on the **receipts_history** dataframe

In [6]:
# Last month receipts history

cond = receipts_history['date'].dt.month == last_month

# Last month new customers

last_month_new_customer = receipts_history[cond]['new_customer'].sum()
print('Last month new customers:',last_month_new_customer)

Last month new customers: 63


## 1.5. CAC (Customer Acquisition Cost)

In [7]:
CAC = (sales_expense + sales_mkt_payroll + last_month_marketing_spendings) / last_month_new_customer
print('Customer acquisition cost:',CAC)

Customer acquisition cost: 1213.968253968254


**It costs $1,213 to acquire each new customer.**

# 2. ARPU (Average Revenue per User)

## 2.1. Total revenue

We calculate **last month total revenue** based on the **receipts_history** dataframe by using data from **receipt_amount** column

In [8]:
# Last month receipts history

cond = receipts_history['date'].dt.month == last_month

# Last month total revenue

last_month_total_revenue = receipts_history[cond]['receipt_amount'].sum()
print('Last month total revenue:',last_month_total_revenue)

Last month total revenue: 83033


## 2.2. Total users

We will calculate **last month total customers** based on the **receipts_history** dataframe

We only take into account the active customers, so we have to count the **unique customers**.

In [9]:
# Last month total users

last_month_total_users = receipts_history[cond]['customer_id'].nunique()
print('Last month total users:',last_month_total_users)

Last month total users: 292


## 2.3. ARPU Calculation

ARPU = Total Revenue / Total number of users

In [10]:
# ARPU

ARPU = last_month_total_revenue / last_month_total_users
print('Average revenue per user:',ARPU)

Average revenue per user: 284.3595890410959


# 3. COGS (Cost of goods sold)

## 3.1. Sever and Software Expenses

We calculate **Sever and Software Expenses** based on the **monthly_expenses** dataframe.

We only take into account the **softwares contribute to make the final product**, which are: 'AWS Hosting', 'Google Cloud Storage', 'Atlassian Jira', 'Slack', 'Zoom'

In [11]:
# Total last month sever and software expenses

last_month_sever_software = last_month_expense[last_month_expense['item'].isin(['AWS Hosting', 'Google Cloud Storage', 'Atlassian Jira', 'Slack', 'Zoom'])]['amount'].sum()
print('Total last month sever and software expenses:',last_month_sever_software)

Total last month sever and software expenses: 15640


## 3.2. Production Expenses

We calcualte the **production expense** based on the **payroll** dataframe

We onnly calculate the **Engineering department** because they directly join in making the product

In [12]:
# Total last month production salary

last_month_production_salary = last_month_payroll[last_month_payroll['department'] == 'Engineering']['paid'].sum()
print('Total last month production salary:',last_month_production_salary)

Total last month production salary: 5200


## 3.3. COGS Calculation

In [13]:
# COGS Calculation

COGS = last_month_sever_software + last_month_production_salary
print('Cost of goods sold:',COGS)

Cost of goods sold: 20840


# 4. Gross Margin

Gross Margin = (Total Revenue - COGS) / Total Revenue

In [14]:
# Gross margin

GM = (last_month_total_revenue - COGS)/last_month_total_revenue
print('Gross margin:',GM)

Gross margin: 0.7490154516878831


# 5. LTV (Customer Lifespan Value)

**LTV = ARPU * Average Lifespan * Gross Margin**

**ARPU**: already calculated

**Average lifespan**: we calculate based on the customer_lifespan_data dataframe by calculating the monthly average lifespan

**Gross Margin**: already calculated


In [15]:
# Average customer

customer_lifespan_data['lifespan'] = customer_lifespan_data['churn_date'] - customer_lifespan_data['start_date']
customer_lifespan_data['lifespan'] = customer_lifespan_data['lifespan'].dt.days
average_lifespan = (customer_lifespan_data['lifespan'].mean())/30
print('Average customer lifespan:',average_lifespan)

Average customer lifespan: 9.841333333333333


In [16]:
# LTV calculation

LTV = ARPU * average_lifespan * GM
print('Customer lifespan value:',LTV)

Customer lifespan value: 2096.102890410959


# 6. LTV/CAC

In [17]:
# LTV/CAC calculation

LTV_CAC = LTV/CAC
print('LTV/CAC:',LTV_CAC)

LTV/CAC: 1.7266537930948014


**LTV/CAC > 1 indicates that the company is generating more revenue from customers than it costs to accquire them. This is a positive sign, suggesting profitability and sustainable growth**